# Lets answer some questions about our data with queries

In [ ]:
import pandas as pd
import sqlite3

First thing to do is to connect to our database

In [ ]:
database = "sqlite_db_pythonsqlite.db"
conn = sqlite3.connect(database)

query = "SELECT name FROM sqlite_master WHERE type='table'"
tables = pd.read_sql_query(query, conn)

print('Lets see the tables dtypes')
for name in tables.name: 
    print(name)
    display(pd.read_sql_query(f"SELECT name, type FROM pragma_table_info('{name}')", conn))
    
    
print('\n\n\n', 'Lets see the tables')
query = '''SELECT *FROM Bookings;'''
query2 = '''SELECT *FROM Facilities;'''
query3 = '''SELECT *FROM Members;'''

display(pd.read_sql_query(query, conn).head())
display(pd.read_sql_query(query2, conn).head())
display(pd.read_sql_query(query3, conn).head())

Lets see the tables dtypes
Bookings


,name,type
0,bookid,int(4)
1,facid,int(1)
2,memid,int(2)
3,starttime,varchar(19)
4,slots,int(2)


Facilities


,name,type
0,facid,int(1)
1,name,varchar(15)
2,membercost,"decimal(2,1)"
3,guestcost,"decimal(3,1)"
4,initialoutlay,int(5)
5,monthlymaintenance,int(4)


Members


,name,type
0,memid,int(2)
1,surname,varchar(17)
2,firstname,varchar(9)
3,address,varchar(39)
4,zipcode,int(5)
5,telephone,varchar(14)
6,recommendedby,varchar(2)
7,joindate,varchar(19)





 Lets see the tables


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


Some of the facilities charge a fee to members, but some do not. Which ones do?

In [ ]:
query1 = '''
SELECT name AS facilities_charging_members
FROM Facilities
WHERE membercost > 0;
'''
pd.read_sql_query(query1, conn)

,facilities_charging_members
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


How many facilities do not charge a fee to members?

In [ ]:
query = '''
SELECT COUNT(name) AS count_of_facilities_NOT_charging_members
FROM Facilities
WHERE membercost = 0;
'''
pd.read_sql_query(query, conn)

,count_of_facilities_NOT_charging_members
0,4


Which facilities charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?

In [ ]:
query = '''
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < .2 * monthlymaintenance
    AND membercost > 0;
'''
pd.read_sql_query(query, conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


Write an SQL query to retrieve the details of facilities with ID 1 and 5.

In [ ]:
query = '''
SELECT *
FROM Facilities
WHERE facid IN (1, 5);
'''
pd.read_sql_query(query, conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. 

In [ ]:
query = '''
SELECT name, monthlymaintenance, 
    (CASE WHEN monthlymaintenance <= 100 THEN 'cheap'
          ELSE 'expensive' END) AS cheap_or_expensive
FROM Facilities;
'''
pd.read_sql_query(query, conn)

,name,monthlymaintenance,cheap_or_expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


You'd like to get the first and last name of the last member(s)
who signed up. 

In [ ]:
query = '''
SELECT firstname AS first, surname AS last
FROM Members
WHERE DATE(joindate) = (SELECT DATE(MAX(joindate))
                  FROM Members);
'''
pd.read_sql_query(query, conn)

,first,last
0,Darren,Smith


Which members have used the tennis courts?

In [ ]:
query = '''
SELECT DISTINCT surname || ', ' || firstname AS member, name
FROM Bookings
LEFT JOIN Members
USING (memid)
LEFT JOIN Facilities
USING (facid)
WHERE name in ('Tennis Court 1', 'Tennis Court 2')
ORDER BY member;
'''
pd.read_sql_query(query, conn)

,member,name
0,"Bader, Florence",Tennis Court 2
1,"Bader, Florence",Tennis Court 1
2,"Baker, Anne",Tennis Court 1
3,"Baker, Anne",Tennis Court 2
4,"Baker, Timothy",Tennis Court 2
5,"Baker, Timothy",Tennis Court 1
6,"Boothe, Tim",Tennis Court 2
7,"Boothe, Tim",Tennis Court 1
8,"Butters, Gerald",Tennis Court 1
9,"Butters, Gerald",Tennis Court 2


Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [ ]:
query = '''
SELECT name, surname || ', ' || firstname AS member, (
    CASE WHEN memid = 0 THEN guestcost
          ELSE membercost
    END * slots) AS cost
FROM Bookings
LEFT JOIN Members
USING (memid)
LEFT JOIN Facilities
USING (facid)
WHERE DATE(starttime) = '2012-09-14'
    AND cost > 30
ORDER BY cost DESC
'''
pd.read_sql_query(query, conn)

,name,member,cost
0,Massage Room 2,"GUEST, GUEST",320.0
1,Massage Room 1,"GUEST, GUEST",160.0
2,Massage Room 1,"GUEST, GUEST",160.0
3,Massage Room 1,"GUEST, GUEST",160.0
4,Tennis Court 2,"GUEST, GUEST",150.0
5,Tennis Court 1,"GUEST, GUEST",75.0
6,Tennis Court 1,"GUEST, GUEST",75.0
7,Tennis Court 2,"GUEST, GUEST",75.0
8,Squash Court,"GUEST, GUEST",70.0
9,Massage Room 1,"Farrell, Jemima",39.6


Produce the same result, but using a subquery.

In [ ]:
query = '''
SELECT member, cost
FROM (SELECT name, surname || ', ' || firstname AS member, (
        CASE WHEN memid = 0 THEN guestcost
              ELSE membercost
        END * slots) AS cost, starttime
    FROM Bookings
    LEFT JOIN Members
    USING (memid)
    LEFT JOIN Facilities
    USING (facid)) AS x
WHERE DATE(starttime) = '2012-09-14'
    AND cost > 30
ORDER BY cost DESC
'''
pd.read_sql_query(query, conn)

,member,cost
0,"GUEST, GUEST",320.0
1,"GUEST, GUEST",160.0
2,"GUEST, GUEST",160.0
3,"GUEST, GUEST",160.0
4,"GUEST, GUEST",150.0
5,"GUEST, GUEST",75.0
6,"GUEST, GUEST",75.0
7,"GUEST, GUEST",75.0
8,"GUEST, GUEST",70.0
9,"Farrell, Jemima",39.6


Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue.

In [ ]:
query = '''
SELECT name, (CASE WHEN total_revenue IS NULL THEN 0 
                   ELSE total_revenue 
              END) AS total_revenue
FROM (
    SELECT name, (
        CASE WHEN memid = 0 THEN guestcost
             ELSE membercost
        END * slots
                 ) AS total_revenue
    FROM Facilities
    LEFT JOIN Bookings
    USING (facid)
    )
WHERE total_revenue < 1000
ORDER BY total_revenue DESC
'''
pd.read_sql_query(query, conn)

,name,total_revenue
0,Massage Room 1,640.0
1,Massage Room 1,640.0
2,Massage Room 1,640.0
3,Massage Room 1,480.0
4,Massage Room 1,480.0
...,...,...
4038,Pool Table,0.0
4039,Pool Table,0.0
4040,Pool Table,0.0
4041,Pool Table,0.0


Produce a report of members and who recommended them in alphabetic surname, firstname order

In [ ]:
query = '''
SELECT l.surname || ', ' || l.firstname AS member, r.surname || ', ' || r.firstname AS recommended_by
FROM Members l 
LEFT JOIN Members r
ON l.recommendedby = r.memid
ORDER BY member;
'''
pd.read_sql_query(query, conn)

,member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


Find the facilities with their usage by member

In [ ]:
query = '''
SELECT name, surname || ', ' || firstname AS member, COUNT(*) AS usage_count
FROM Bookings 
LEFT JOIN Facilities
USING(facid)
LEFT JOIN Members
USING(memid)
WHERE memid > 0
GROUP BY name, member;
'''
pd.read_sql_query(query, conn)

,name,member,usage_count
0,Badminton Court,"Bader, Florence",9
1,Badminton Court,"Baker, Anne",10
2,Badminton Court,"Baker, Timothy",7
3,Badminton Court,"Boothe, Tim",12
4,Badminton Court,"Butters, Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith, Darren",19
198,Tennis Court 2,"Smith, Jack",1
199,Tennis Court 2,"Smith, Tracy",2
200,Tennis Court 2,"Stibbons, Ponder",31


Find the facilities usage by month

In [ ]:
query = '''
SELECT name, strftime('%m', starttime) AS month, COUNT(*) AS usage_count
FROM Bookings 
LEFT JOIN Facilities
USING(facid)
LEFT JOIN Members
USING(memid)
WHERE memid > 0
GROUP BY name, month;
'''
pd.read_sql_query(query, conn)

,name,month,usage_count
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
